In [ ]:
import os

isKaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE','')
if isKaggle:
    !pip install -Uqq fastai duckduckgo_search

define a function to search images

In [ ]:
from fastcore.all import *
from duckduckgo_search import DDGS
#return a list of url using the search term
def search_images(term,max_images = 30):
    print(f"Searching for '{term}'")
    with DDGS() as ddgs:
         ddgs_images_gen = iter(ddgs.images(term))
         cnt = 0;
         ddgs_image_list = []
         while(cnt < max_images):
             image = next(ddgs_images_gen)
             ddgs_image_list.append(image.get('image'))
             cnt = cnt+1
         return ddgs_image_list        
         
    

now print url of an image

In [ ]:
urls = search_images('grizzly bear')
urls[0]

download a url to take a look at the image

In [ ]:
from fastdownload import download_url
dest = 'bear.jpg' #saved file name
download_url(urls[0],dest)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
from time import sleep
bear_types = 'grizzly', 'black', 'teddy'
path = Path('bears')
for o in bear_types:
    dest = path/o
    dest.mkdir(exist_ok=True,parents = True)
    download_images(dest,urls = search_images(f'{o} bear photos'))
    sleep(10)
    resize_images(path/o,max_size=400,dest=path/o)

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
bears = DataBlock((ImageBlock,CategoryBlock),
                 get_items = get_image_files,
                 splitter = RandomSplitter(valid_pct = 0.2,seed =42),
                 get_y = parent_label,
                 item_tfms = Resize(128))
bears = bears.new(item_tfms =RandomResizedCrop(128, min_scale=0.3))
dls = bears.dataloaders(path)
dls.show_batch(max_n=6)

In [ ]:
learner = vision_learner(dls,resnet18,metrics = error_rate)
learner.fine_tune(4)

In [ ]:
is_bear,_,probs = learner.predict(PILImage.create('bear.jpg'))
print(f"This is a : {is_bear}")


In [ ]:
interp = ClassificationInterpretation.from_learner(learner)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5,nrows=1)

We trained our model and now we can use it to clean some data, to do that we need to see the pictures it guessed wrong or didnt guess confidently 

In [ ]:
from fastai.vision.widgets import *
cleaner = ImageClassifierCleaner(learner)
cleaner

In [ ]:
for idx in cleaner.delete():
     cleaner.fns[idx].unlink()
for idx,cat in cleaner.change():
    shutil.move(str(cleaner.fns[idx]),path/cat)

In [71]:
#saving the model
learner.export()

In [ ]:
path = Path()
path.ls(file_exts = '.pkl')